In [4]:
import numpy as np
import os

In [65]:
mainpath = '/Users/aleex/DSE/Machine Learning/Project/toyData/'

outpath = '/Users/aleex/DSE/Machine Learning/Project/label/'
if not os.path.exists(outpath):
    os.makedirs(outpath)
    


In [15]:
#To be modified on both Goes16 and 17
filenames = []
for root, dirs, files in os.walk(mainpath):
    for file in files:
        if file.endswith(".npy"):
             filenames.append(os.path.join(root, file))
print(filenames)

['/Users/aleex/DSE/Machine Learning/Project/toyData/OR_ABI-L1b-RadF-M6C07_G16_s20192210000548_e20192210010268_c20192210010328.npy', '/Users/aleex/DSE/Machine Learning/Project/toyData/OR_ABI-L1b-RadF-M6C07_G16_s20192220000555_e20192220010275_c20192220010333.npy']


In [64]:
#for file in filenames:
#THIS FUNCTION ONLY RUNS ON THE TWO TOY DATA, PIPELINE NEEDS TO BE MODIFIED FOR THE WHOLE COLLECTION 
goes1 = np.load(filenames[0])
goes2 = np.load(filenames[1])

n1 = len(goes1)*len(goes1[0])
n2 = len(goes2)*len(goes2[0])

std1 = np.std(goes1)
std2 = np.std(goes2)

joint_std = np.sqrt(std1**2/n1 + std2**2/n2)
delta = abs(goes1 - goes2)
label = delta//joint_std
label = label.astype(int) + 1
print(delta)
print(joint_std)
print(label)

[[0.00312865 0.01720786 0.05005926 ... 0.20492995 0.1110689  0.02033657]
 [0.00312865 0.05005926 0.00312871 ... 0.16582125 0.057881   0.00938612]
 [0.04693049 0.00312871 0.00312871 ... 0.1329698  0.0078218  0.02502966]
 ...
 [0.7712251  0.77904683 0.77904683 ... 0.18302906 0.1689499  0.14861333]
 [0.7524529  0.7978191  0.77904683 ... 0.12514806 0.0829106  0.07978189]
 [0.7524529  0.7962547  0.7978191  ... 0.0250296  0.04223746 0.01720786]]
0.00023241622509243654
[[  14   75  216 ...  882  478   88]
 [  14  216   14 ...  714  250   41]
 [ 202   14   14 ...  573   34  108]
 ...
 [3319 3352 3352 ...  788  727  640]
 [3238 3433 3352 ...  539  357  344]
 [3238 3426 3433 ...  108  182   75]]


In [68]:
out = outpath + 'label1.txt'
np.savetxt(out, label, fmt='%10.f')